---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # A university town is a city which has a high percentage of university students 
    # compared to the total population of the city.
    college_towns = pd.read_table("university_towns.txt", header=None)
    # http://stackoverflow.com/questions/41457322/pandas-rearranging-a-data-frame/41458629#41458629
    college_towns['State'] = college_towns[0].str.extract('(.*)\[edit\]', expand=False).ffill()
    college_towns['RegionName'] = college_towns[0].str.replace(r'\s+\(.+$', '')
    college_towns = college_towns[~college_towns[0].str.contains('\[edit\]')].reset_index(drop=True).drop(0, axis=1)
    
    return college_towns

## Read In + Clean GDP Data

In [5]:
# Read in GDP over time data
GDP = pd.read_excel("gdplev.xls", skiprows={0,1,2,3,4,6,7})
GDP.rename(columns={'Unnamed: 0': 'Year', 'Unnamed: 3': 'Quarter', 'Unnamed: 4': 'Year_Quarter', 'GDP in billions of current dollars.1': 'GDP_Billions'}, inplace=True)

# use the chained value in 2009 dollars
GDP.drop({'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'Unnamed: 7', 'GDP in billions of chained 2009 dollars.1'},inplace=True,axis=1)

# Split Column into Year and Quarter
# http://stackoverflow.com/questions/14745022/pandas-dataframe-how-do-i-split-a-column-into-two
GDP['Year'] = GDP['Year_Quarter'].str.split('q', 1).str[0].astype(int)
GDP['Quarter'] = GDP['Year_Quarter'].str.split('q', 1).str[1].astype(int)
# For this assignment, only look at GDP data from the first quarter of 2000 onward.
GDP = GDP[GDP['Year'] >= 2000]

In [6]:
# # http://pandas.pydata.org/pandas-docs/stable/visualization.html
# import matplotlib.pyplot as plt

# GDP.plot(x='Year_Quarter', y='GDP_Billions')
# plt.show()

In [7]:
# A recession is defined as starting with two consecutive quarters of GDP decline, 
# and ending with two consecutive quarters of GDP growth.

In [12]:
# http://stackoverflow.com/questions/41361537/find-two-consecutive-quarters-of-gdp-decline-and-ending-with-two-consecutive-qu
# given dataframe GDP, returns the start_index and end_index for recession, as well as the index for the lowest point (bottom)

def get_recession(): 
    start_index = 0
    end_index = 0
    bottom = (0, float("inf"))
    recession = False # save current status of economy, assuming that we did not begin with a recession

    # start from 2nd quarter, so comparison begins with index i-1
    for i in range(1, len(GDP) - 1):
        # print(GDP['Year_Quarter'].iloc[i], GDP['GDP_Billions'].iloc[i])
        if not recession and (GDP['GDP_Billions'].iloc[i-1] > GDP['GDP_Billions'].iloc[i] > GDP['GDP_Billions'].iloc[i+1]):
            recession = True
            start_index = i-1
        elif recession:
            if (GDP['GDP_Billions'].iloc[i-1] < GDP['GDP_Billions'].iloc[i] < GDP['GDP_Billions'].iloc[i+1]):
                recession = False
                end_index = i+1 
            # to find the bottom, find the lowest point within the recession
            if GDP['GDP_Billions'].iloc[i] < bottom[1]:
                bottom = (i, GDP['GDP_Billions'].iloc[i])
            
    return (start_index, end_index, bottom)

## Use get_recession() function

In [13]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    start_index = get_recession()[0]
    start = GDP['Year_Quarter'].iloc[start_index]
    return start

In [14]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    end_index = get_recession()[1]
    end = GDP['Year_Quarter'].iloc[end_index]
    return end

In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    bottom_index = get_recession()[2][0]
    bottom = GDP['Year_Quarter'].iloc[bottom_index]
    return bottom

## Convert Housing Data to Quarters
* http://stackoverflow.com/questions/40728309/pandas-goup-and-rename-dataframe-columns-names
* http://stackoverflow.com/questions/40497199/how-to-convert-monthly-data-to-quarterly-in-pandas
* https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/CSgStATMEee9mg5CyCleJA
* http://stackoverflow.com/questions/41661451/pandas-compute-the-half-yearly-average-of-monthly-collected-house-sales

A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.

In [11]:
# Read in homes data (at city level)
homes = pd.read_csv("City_Zhvi_AllHomes.csv")
# map State abbreviations to full name
homes['State'] = homes['State'].map(lambda x: states[x])
# homes.head()

In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = homes
    # Step 1. separate columns we want to resample
    # only want columns from 2000-1 to 2016-8
    tdf = df.filter(regex=("20.*"))
    # Step 2. change the column names type to datetime
    tdf.columns = pd.to_datetime(tdf.columns)
    # Step 3. Perform the following on tdf: resample then take the mean on the column by setting 
    # resample's parameter axis to 1. This will generate a dataframe with the same number of rows as tdf 
    # and the number to columns will be the ceiling (len(tdf.columns))/3
    mdf = tdf.resample('Q',axis=1).mean() 
    # Step 4. Change the column names back to string. This is a simple operation given that datatime types 
    # have the member attributes year and quarter.
    mdf.columns = mdf.columns.map(lambda x: '{:}q{:}'.format(x.year, x.quarter))
    # Step 5. Once you're at this stage you only need to add two columns to mdf, 
    # the column with state names changed according the states dictionary and the regionNames.
    # Then, set the index to ['state','RegionName'].
    mdf['RegionName'] = df['RegionName']
    mdf['State'] = df['State']
    mdf.set_index(['State', 'RegionName'], inplace=True)
    
    return mdf

## Run ttest_ind
* https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/iCSsLhrvEee0thK6z3XwTA

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

Once you have the two groups, to test the hypothesis is null (invalid), we run the ttest on the price ratio column of each group. Now the result of the ttest pvalue determines if the null hypothesis is true or not.

The term "null hypothesis" is a general statement or default position that there is no relationship between two measured phenomena.

If pvalue <0.01 we reject the null hypothesis and hence it's possible that there is a relationship between {housing prices being affected by recession} and {being located in a university town or not a university town}

If pvalue >0.01 we cannot reject the null hypothesis and hence there is no relationship between {housing prices being affected by recession} and {being located in a university town or not a university town}

In [13]:
def get_quarter_before_recession_start():
    start_index = get_recession()[0]
    qrbr = GDP['Year_Quarter'].iloc[start_index-1]
    return qrbr

In [14]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Step 1. Inputs to run_ttest()
    hdf = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()

    # Step 2. Calculate Price Ratio
    # price_ratio=quarter_before_recession/recession_bottom
    qrbr = get_quarter_before_recession_start() # i.e if start='2001q1' then qrbr='2000q4'
    hdf['PriceRatio'] = hdf[qrbr].div(hdf[rec_bottom])
    
    # Step 3. Separate the university towns from non_university towns in hdf
    # To test the hypothesis (University towns have their mean housing prices less effected by recessions)
    # http://stackoverflow.com/questions/38877766/converting-pandas-dataframe-into-list-of-tuples-with-index
    tuple_list = [tuple(x) for x in ul.to_records(index=False)]
    university_towns = hdf.loc[tuple_list]
    non_university_towns = hdf.loc[~hdf.index.isin(tuple_list)]
    
    # Step 4. Run ttest
    answer = ttest_ind(university_towns['PriceRatio'], non_university_towns['PriceRatio'], nan_policy='omit') 
    # different=True if the t-test is True at a p<0.01
    different = answer.pvalue < 0.01
    # The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind()
    p = answer.pvalue
    # The value for better should be either "university town" or "non-university town"
    # depending on which has a lower mean price ratio
    x = university_towns['PriceRatio'].mean()
    y = non_university_towns['PriceRatio'].mean()
    better = "university town" if x < y else "non-university town"
    
    return (different, p, better)